In [1]:
import pandas as pd
import numpy as np
from scipy.stats import norm

In [2]:
# --- carregar base
df_sim = pd.read_csv('./base_suja/base_sim_dofet_suja.csv')

In [7]:
df_sim.columns

Index(['data_obito', 'ano_obito', 'TIPOBITO', 'ocor_MUNNOMEX', 'res_MUNNOMEX',
       'ocor_CAPITAL', 'res_CAPITAL', 'ocor_REGIAO', 'res_REGIAO',
       'ocor_SIGLA_UF', 'res_SIGLA_UF', 'IDADEMAE', 'idademae_faixa',
       'ESCMAE2010', 'escolaridade_mae', 'OBITOGRAV', 'GRAVIDEZ',
       'tipo_gravidez', 'SEMAGESTAC', 'idade_gestacao_faixa', 'SEXO',
       'def_sexo', 'PESO', 'peso_faixa', 'OBITOPARTO', 'def_obito_parto',
       'CAUSABAS', 'causabas_capitulo', 'causabas_categoria', 'causabas_grupo',
       'causabas_subcategoria', 'FLAG_BASE'],
      dtype='object')

In [ ]:
df_sim['cat_peso_calc_2'] = ['PIG' if i=='PIG' else 'NAO_PIG' for i in df_sim['cat_peso_calc']]

In [ ]:
df_sim_agreg_regiao = pd.DataFrame(
    df_sim.groupby(['cat_periodo_nasc', 'cat_peso_calc_2', 'ocor_REGIAO']).agg(Tamanho=('data_obito', 'size'))
).reset_index()

In [ ]:
df_sim_agreg_brasil = pd.DataFrame(
    df_sim.groupby(['cat_periodo_nasc', 'cat_peso_calc_2']).agg(Tamanho=('data_obito', 'size'))
).reset_index()

In [ ]:
df_sinasc = pd.read_csv('./base_suja/base_sinasc_suja.csv')

In [ ]:
df_sinasc['cat_peso_calc_2'] = ['PIG' if i=='PIG' else 'NAO_PIG' for i in df_sinasc['cat_peso_calc']]

In [ ]:
df_sinasc_agreg_regiao = pd.DataFrame(
    df_sinasc.groupby(['cat_periodo_nasc', 'cat_peso_calc_2', 'nasc_REGIAO']).agg(Tamanho=('data_nasc', 'size'))
).reset_index()

In [ ]:
df_sinasc_agreg_brasil = pd.DataFrame(
    df_sinasc.groupby(['cat_periodo_nasc', 'cat_peso_calc_2']).agg(Tamanho=('data_nasc', 'size'))
).reset_index()

In [ ]:
df_sinasc_agreg_regiao

In [ ]:
# ------------------------------------------------------------
# 0) AJUSTES E MAPAS
# ------------------------------------------------------------

# Mapa opcional para nomes "bonitos" das 4 combinações usuais
pretty_map = {
    ('pre_termo', 'PIG'): 'Preterm and SGA',
    ('pre_termo', 'NAO_PIG'): 'Preterm non-SGA',
    ('termo', 'PIG'): 'Term and SGA',
    ('termo', 'NAO_PIG'): 'Term and non-SGA'
}

def label_cat(row):
    t = (row['cat_periodo_nasc'], row['cat_peso_calc_2'])
    return pretty_map.get(t, f"{row['cat_periodo_nasc']} & {row['cat_peso_calc_2']}")

In [ ]:
# ------------------------------------------------------------
# 1) MISSING VALUES (a partir das bases não agregadas)
#    -> altere os nomes se necessário
# ------------------------------------------------------------
def summarize_missing(df, vars_):
    out = df[vars_].isna().sum().to_frame('n_missing')
    out['perc_missing'] = 100 * out['n_missing'] / len(df)
    return out

miss_sinasc = summarize_missing(
    df_sinasc,
    ['cat_periodo_nasc', 'cat_peso_calc_2', 'nasc_REGIAO']
)
miss_sim = summarize_missing(
    df_sim,
    ['cat_periodo_nasc', 'cat_peso_calc_2', 'ocor_REGIAO']
)

print("Missing SINASC (variáveis usadas):")
print(miss_sinasc)
print("\nMissing SIM (variáveis usadas):")
print(miss_sim)


In [ ]:
# ------------------------------------------------------------
# 2) PREPARA CONTAGENS DO SINASC (REGIÃO E BRASIL)
# ------------------------------------------------------------
# Região: contagem por categoria + total da região
sinasc_reg = df_sinasc_agreg_regiao.copy()
sinasc_reg['SVN_cat'] = sinasc_reg.apply(label_cat, axis=1)

# total por REGIÃO (denominador)
denom_reg = (
    sinasc_reg.groupby('nasc_REGIAO')['Tamanho']
    .sum()
    .rename('n_total_reg')
    .reset_index()
)

# contagem por REGIÃO x CATEGORIA (numerador)
num_reg = (
    sinasc_reg
    .groupby(['nasc_REGIAO', 'SVN_cat'])['Tamanho']
    .sum()
    .rename('n_cat_reg')
    .reset_index()
)

# Brasil: contagem por categoria e total
sinasc_br = df_sinasc_agreg_brasil.copy()
sinasc_br['SVN_cat'] = sinasc_br.apply(label_cat, axis=1)

n_total_br = sinasc_br['Tamanho'].sum()
num_br = (
    sinasc_br.groupby('SVN_cat')['Tamanho']
    .sum()
    .rename('n_cat_br')
    .reset_index()
)
num_br['n_total_br'] = n_total_br
num_br['p_br'] = num_br['n_cat_br'] / num_br['n_total_br']

In [ ]:
num_reg

In [ ]:
# ------------------------------------------------------------
# 3) FUNÇÕES AUXILIARES
# ------------------------------------------------------------
def prev_ci_wald(n_cat, n_total):
    """Retorna string 'xx.xx (li-hi)' para % por 100 NV com IC95% (Wald)."""
    if n_total == 0 or pd.isna(n_total):
        return np.nan
    p = n_cat / n_total
    se = np.sqrt(p * (1 - p) / n_total) if n_total > 0 else np.nan
    lo = max(p - 1.96 * se, 0)
    hi = min(p + 1.96 * se, 1)
    return f"{p*100:.2f} ({lo*100:.2f}-{hi*100:.2f})"

def pvalue_reg_vs_br(n_cat_reg, n_total_reg, p_br):
    """Teste z bicaudal H0: p_reg = p_br (comparação região x Brasil)."""
    if n_total_reg == 0 or pd.isna(n_total_reg):
        return np.nan
    p_reg = n_cat_reg / n_total_reg
    se_h0 = np.sqrt(p_br * (1 - p_br) / n_total_reg)
    if se_h0 == 0:
        return np.nan
    z = (p_reg - p_br) / se_h0
    return 2 * (1 - norm.cdf(abs(z)))


In [ ]:
# ------------------------------------------------------------
# 4) MONTA TABELA POR REGIÃO + p-VALUES
# ------------------------------------------------------------
# Garante que todas as categorias existam em todas as regiões (preenche 0)
all_cats = sorted(num_br['SVN_cat'].unique().tolist())
all_regs = sorted(denom_reg['nasc_REGIAO'].unique().tolist())
grid = pd.MultiIndex.from_product([all_regs, all_cats],
                                  names=['nasc_REGIAO','SVN_cat']).to_frame(index=False)

num_reg_full = grid.merge(num_reg, on=['nasc_REGIAO','SVN_cat'], how='left')
num_reg_full['n_cat_reg'] = num_reg_full['n_cat_reg'].fillna(0).astype(int)
num_reg_full = num_reg_full.merge(denom_reg, on='nasc_REGIAO', how='left')

# junta p_br por categoria
num_reg_full = num_reg_full.merge(num_br[['SVN_cat','p_br']], on='SVN_cat', how='left')

# calcula % (IC) e p-value
num_reg_full['percent_ic'] = num_reg_full.apply(
    lambda r: prev_ci_wald(r['n_cat_reg'], r['n_total_reg']), axis=1
)
num_reg_full['p_value'] = num_reg_full.apply(
    lambda r: pvalue_reg_vs_br(r['n_cat_reg'], r['n_total_reg'], r['p_br']), axis=1
)

# formata p-value
def fmt_p(p):
    if pd.isna(p): 
        return ''
    if p < 0.001:
        return '<0.001'
    return f'{p:.3f}'

num_reg_full['p_value_fmt'] = num_reg_full['p_value'].apply(fmt_p)


In [ ]:
# ------------------------------------------------------------
# 5) REORGANIZA NO LAYOUT DA SUA TABELA
#    (3 blocos com p-value + 1 bloco final sem p-value para Term non-SGA)
# ------------------------------------------------------------
def is_term_non_sga(cat_name):
    return cat_name.lower() in ['term and non-sga', 'term & non-sga', 'term non-sga']

wide_parts = []
for cat in all_cats:
    sub = (num_reg_full[num_reg_full['SVN_cat'] == cat]
           .loc[:, ['nasc_REGIAO','percent_ic','p_value_fmt']]
           .rename(columns={'nasc_REGIAO':'Region',
                            'percent_ic': f'{cat} % (95% CI)',
                            'p_value_fmt': f'{cat} p-value'}))
    # se for Term and non-SGA, deixa p-value vazio
    if is_term_non_sga(cat):
        sub[f'{cat} p-value'] = ''
    wide_parts.append(sub)

tabela_reg = wide_parts[0]
for part in wide_parts[1:]:
    tabela_reg = tabela_reg.merge(part, on='Region', how='left')

# adiciona N total por região
tabela_reg = tabela_reg.merge(denom_reg.rename(columns={'nasc_REGIAO':'Region',
                                                        'n_total_reg':'N nascidos vivos (região)'}),
                              on='Region', how='left')

# ordena colunas (cat ordem = all_cats)
col_order = ['Region']
for cat in all_cats:
    col_order += [f'{cat} % (95% CI)', f'{cat} p-value']
col_order += ['N nascidos vivos (região)']
tabela_reg = tabela_reg[col_order]

In [ ]:
# ------------------------------------------------------------
# 6) LINHA DO BRASIL
# ------------------------------------------------------------
linha_br = {'Region': 'Brasil',
            'N nascidos vivos (região)': int(n_total_br)}
for cat in all_cats:
    n_cat_br = int(num_br.loc[num_br['SVN_cat'] == cat, 'n_cat_br'])
    linha_br[f'{cat} % (95% CI)'] = prev_ci_wald(n_cat_br, n_total_br)
    linha_br[f'{cat} p-value'] = ''  # referência nacional

tabela_final = pd.concat([tabela_reg, pd.DataFrame([linha_br])], ignore_index=True)

In [ ]:
# ------------------------------------------------------------
# 7) RESULTADO
# ------------------------------------------------------------
print("\nTabela de Prevalência por 100 NV (% e IC95%) e p-valor (Região vs Brasil):")
print(tabela_final)

In [ ]:
tabela_final

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import norm

# =========================
# Parâmetros
# =========================
SCALE = 1000.0  # use 100.0 se preferir por 100 NV
DECIMALS = 2

# =========================
# 1) Rotulagem das categorias SVN
#    (ajuste os rótulos se seus nomes forem diferentes)
# =========================
pretty_map = {
    ('Preterm', 'SGA'): 'Preterm and SGA',
    ('Preterm', 'non-SGA'): 'Preterm non-SGA',
    ('Term', 'SGA'): 'Term and SGA',
    ('Term', 'non-SGA'): 'Term and non-SGA'
}

def label_cat(row):
    t = (row['cat_periodo_nasc'], row['cat_peso_calc'])
    return pretty_map.get(t, f"{row['cat_periodo_nasc']} & {row['cat_peso_calc']}")

def is_term_non_sga(cat_name: str) -> bool:
    name = (cat_name or "").lower()
    return 'term' in name and 'non' in name and 'sga' in name

# =========================
# 2) Preparar tabelas REGIONAIS (numerador SIM e denominador SINASC)
# =========================
sim_reg = df_sim_agreg_regiao.copy()
sim_reg['SVN_cat'] = sim_reg.apply(label_cat, axis=1)
sim_reg = sim_reg.rename(columns={'ocor_REGIAO': 'REGIAO', 'Tamanho': 'obitos_reg_cat'})

sinasc_reg = df_sinasc_agreg_regiao.copy()
sinasc_reg['SVN_cat'] = sinasc_reg.apply(label_cat, axis=1)
sinasc_reg = sinasc_reg.rename(columns={'nasc_REGIAO': 'REGIAO', 'Tamanho': 'nasc_reg_cat'})

# Grade completa (garante 0 onde faltar)
all_regs = sorted(sinasc_reg['REGIAO'].unique().tolist())
all_cats = sorted(
    set(sim_reg['SVN_cat'].unique()).union(set(sinasc_reg['SVN_cat'].unique()))
)
grid = pd.MultiIndex.from_product([all_regs, all_cats], names=['REGIAO', 'SVN_cat']).to_frame(index=False)

# Mescla numerador e denominador
reg = (grid
       .merge(sim_reg[['REGIAO','SVN_cat','obitos_reg_cat']], on=['REGIAO','SVN_cat'], how='left')
       .merge(sinasc_reg[['REGIAO','SVN_cat','nasc_reg_cat']], on=['REGIAO','SVN_cat'], how='left'))

reg['obitos_reg_cat'] = reg['obitos_reg_cat'].fillna(0).astype(int)
reg['nasc_reg_cat'] = reg['nasc_reg_cat'].fillna(0).astype(int)

# =========================
# 3) Preparar tabelas BRASIL (numerador SIM e denominador SINASC)
# =========================
sim_br = df_sim_agreg_brasil.copy()
sim_br['SVN_cat'] = sim_br.apply(label_cat, axis=1)
sim_br = (sim_br.groupby('SVN_cat', as_index=False)['Tamanho'].sum()
                .rename(columns={'Tamanho': 'obitos_br_cat'}))

sinasc_br = df_sinasc_agreg_brasil.copy()
sinasc_br['SVN_cat'] = sinasc_br.apply(label_cat, axis=1)
sinasc_br = (sinasc_br.groupby('SVN_cat', as_index=False)['Tamanho'].sum()
                      .rename(columns={'Tamanho': 'nasc_br_cat'}))

br = sim_br.merge(sinasc_br, on='SVN_cat', how='outer').fillna(0)
br['obitos_br_cat'] = br['obitos_br_cat'].astype(int)
br['nasc_br_cat'] = br['nasc_br_cat'].astype(int)
br['p_br'] = np.where(br['nasc_br_cat'] > 0, br['obitos_br_cat'] / br['nasc_br_cat'], np.nan)

# =========================
# 4) Funções: prevalência + IC95% (Wald) e p-value (z teste, bicaudal)
# =========================
def prev_ci_wald(n_events, n_total, scale=SCALE, k=1.96):
    if n_total <= 0:
        return np.nan, np.nan, np.nan, ""
    p = n_events / n_total
    se = np.sqrt(p * (1 - p) / n_total)
    lo = max(p - k * se, 0)
    hi = min(p + k * se, 1)
    txt = f"{(p*scale):.{DECIMALS}f} ({(lo*scale):.{DECIMALS}f}-{(hi*scale):.{DECIMALS}f})"
    return p, lo, hi, txt

def pvalue_region_vs_br(n_events_reg, n_total_reg, p_br):
    if (n_total_reg <= 0) or (p_br is None) or np.isnan(p_br):
        return np.nan
    # z sob H0: p_reg = p_br
    p_reg = n_events_reg / n_total_reg
    se_h0 = np.sqrt(p_br * (1 - p_br) / n_total_reg)
    if se_h0 == 0:
        return np.nan
    z = (p_reg - p_br) / se_h0
    return 2 * (1 - norm.cdf(abs(z)))

def fmt_p(p):
    if pd.isna(p):
        return ''
    return '<0.001' if p < 0.001 else f'{p:.3f}'

# =========================
# 5) Calcula por REGIÃO x CATEGORIA
# =========================
reg = reg.merge(br[['SVN_cat','p_br']], on='SVN_cat', how='left')

reg_calc = []
for (regiao, cat), row in reg.set_index(['REGIAO','SVN_cat']).iterrows():
    n_deaths = int(row['obitos_reg_cat'])
    n_births = int(row['nasc_reg_cat'])
    p_reg, lo, hi, prev_txt = prev_ci_wald(n_deaths, n_births, scale=SCALE)
    pval = pvalue_region_vs_br(n_deaths, n_births, row['p_br'])
    pval_fmt = '' if is_term_non_sga(cat) else fmt_p(pval)
    reg_calc.append({
        'Region': regiao,
        'SVN_cat': cat,
        'Prev % (IC95%)': prev_txt,   # escala *p.ex.* 1.000 NV
        'p-value': pval_fmt,
        'N nascidos vivos (cat, região)': n_births,
        'Óbitos (cat, região)': n_deaths
    })

reg_calc = pd.DataFrame(reg_calc)

# =========================
# 6) Linha do BRASIL por categoria
# =========================
br_calc = []
for _, r in br.iterrows():
    n_deaths = int(r['obitos_br_cat'])
    n_births = int(r['nasc_br_cat'])
    _, _, _, prev_txt = prev_ci_wald(n_deaths, n_births, scale=SCALE)
    br_calc.append({
        'Region': 'Brasil',
        'SVN_cat': r['SVN_cat'],
        'Prev % (IC95%)': prev_txt,
        'p-value': '',  # referência nacional
        'N nascidos vivos (cat, região)': n_births,
        'Óbitos (cat, região)': n_deaths
    })
br_calc = pd.DataFrame(br_calc)

long_final = pd.concat([reg_calc, br_calc], ignore_index=True)

# =========================
# 7) Converte para formato LARGO (uma coluna por categoria)
# =========================
# Se quiser seguir o mesmo layout anterior (cada categoria com % e p-value lado a lado):
cats_order = [c for c in pretty_map.values() if c in all_cats] + [c for c in all_cats if c not in pretty_map.values()]

wide_parts = []
for cat in cats_order:
    sub = (long_final[long_final['SVN_cat'] == cat]
            .loc[:, ['Region','Prev % (IC95%)','p-value']]
            .rename(columns={'Prev % (IC95%)': f'{cat} % (95% CI)',
                             'p-value': f'{cat} p-value'}))
    # manter p-value em branco para Term and non-SGA
    if is_term_non_sga(cat):
        sub[f'{cat} p-value'] = ''
    wide_parts.append(sub)

tabela = wide_parts[0]
for part in wide_parts[1:]:
    tabela = tabela.merge(part, on='Region', how='outer')

# Totais por região (somando nascimentos em todas as categorias)
tot_reg = (sinasc_reg.groupby('REGIAO')['nasc_reg_cat'].sum()
                    .rename('N nascidos vivos (região)').reset_index()
                    .rename(columns={'REGIAO':'Region'}))
tabela = tabela.merge(tot_reg, on='Region', how='left')

# Ordena regiões alfabeticamente, Brasil por último
tabela = tabela.sort_values(by=['Region'])
if 'Brasil' in tabela['Region'].values:
    br_row = tabela[tabela['Region'] == 'Brasil']
    tabela = pd.concat([tabela[tabela['Region'] != 'Brasil'], br_row], ignore_index=True)

# =========================
# 8) Resultado
# =========================
print("Prevalência de óbitos por categoria SVN (por {:.0f} NV), IC95% e p-valor (Região vs Brasil):".format(SCALE))
print(tabela)
# Se quiser salvar para Excel/CSV:
# tabela.to_excel('prevalencia_obitos_por_categoria_regiao.xlsx', index=False)
# tabela.to_csv('prevalencia_obitos_por_categoria_regiao.tsv', sep='\t', index=False)
